In [2]:
import pandas as pd
import numpy as np
import glob
import nltk
import json
import tensorflow as tf

import re
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
from pprint import pprint
from nltk.corpus import stopwords

In [20]:
df = pd.read_csv('Political.csv' , sep='\t' , encoding='utf-8')
articles = pd.DataFrame(df)
articles.shape

(24755, 3)

In [27]:
# Inspecting some of the articles
for i in range(10, 15):
    print("Article: #", i)
    print(articles.Summary[i])
    print()
    print(articles.Text[i])
    print()
    print()

Article: # 10
The Children's Investment Fund Foundation | Opinion - Conservative

(Before It's News)
The Children's Investment Fund Foundation https://en.wikipedia.org/wiki/The_Children's_Investment_Fund_Foundation The Children's Investment Fund Foundation (UK) ( CIFF ) is a large charitable organization headquartered in London with offices in Nairobi and New Delhi. It is a registered charity in England & Wales and in 2012–13 had an income of £100 million and spending of £98 million. [1] With assets of £2.2 billion, it is the 4th largest charity in the UK . Activities The Clinton Foundation lists CIFF among 7 organisations from which it received more than US$25 million. Children's Investment Fund Foundation Mark Malloch-Brown is a trustee at the Children's Investment Fund Foundation , an advisory council member for the Acumen Fund , a co-chair for the International Crisis Group , a minister of state for the United Kingdom , and the vice chairman for Refugees International . Note: Andre

PREPROCESSING THE DATA

In [28]:
# A list of common contractions used in articles in English
contractions = { 
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [36]:
def clean_text(text, remove_stopwords = True, is_summary = False):
    ''' This function is used to remove unwanted characters, stopwords
        and format the text to create fewer nulls word embeddings '''
    
    #Convert all words to lower case
    text = text.lower()
    
    #Replace contractions with their grammatically forms 
    if True:
        text = text.split()
        new_text = []
        
        for word in text:    
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    #Format the words formed and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r]', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    if is_summary == False:
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
        
    ''' This option is provided so that the function can be reused for text and summary cleaning
        Keeping stopwords in the generated summary is to make them sound like natural phrases '''

    
    
    n_t = text.split()
    n_t = n_t[:120]
    
    return " ".join(n_t)

In [37]:
#Clean the summaries and texts

article_count = 0
clean_summaries = []
for summary in articles.Summary:        
    clean_summaries.append( clean_text(summary, remove_stopwords=False, is_summary=True) )
    article_count +=1 
    if article_count==7501:
        break
    #lean_summaries.append(summary)
    
print("Summaries are completely cleaned.")

article_count = 0
clean_texts = []
for text in articles.Text:
    clean_texts.append( clean_text(text) )
    article_count +=1 
    if article_count==7501:
        break
print("Texts are completely cleaned.")

Summaries are completely cleaned.
Texts are completely cleaned.


In [38]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned as per needed
for i in range(10,15):
    print("Cleaned Article: #",i)
    print(clean_summaries[i])
    print()
    print(clean_texts[i])
    print()
    print()

Cleaned Article: # 10
the children s investment fund foundation | opinion - conservative

news children investment fund foundation https en wikipedia org wiki children investment fund foundation children investment fund foundation uk ciff large charitable organization headquartered london offices nairobi new delhi registered charity england wales 2012–13 income £100 million spending £98 million 1 assets £2 2 billion 4th largest charity uk activities clinton foundation lists ciff among 7 organisations received us 25 million children investment fund foundation mark malloch brown trustee children investment fund foundation advisory council member acumen fund co chair international crisis group minister state united kingdom vice chairman refugees international note andrea soros director acumen fund george soros’s daughter george soros andrea soros’s father board member international crisis group founder chairman open society foundations director emeritus refugees international chairman


C

In [39]:
# Count the number of occurrences of each word in the text

def count_words(count_dict, text):
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [40]:
# Determine the collective size of the vocabulary formed
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 50231


In [42]:
# Load Conceptnet Numberbatch's (CN) embeddings
''' (https://github.com/commonsense/conceptnet-numberbatch) '''

embeddings_index = {}
with open('./numberbatch-en.txt', encoding='utf-8') as cn:
    for line in cn:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

    
print('Total number of word embeddings:', len(embeddings_index))

Total number of word embeddings: 417195


In [44]:
# And hence find the number of words that are missing from CN, but are used more than a threshold number of times

missing_words = 0
threshold = 10
mw = []

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            mw.append(word)
            missing_words += 1
            
missing_percentage = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from the vocabulary: {}%".format(missing_percentage))


Number of words missing from CN: 645
Percent of words that are missing from the vocabulary: 1.28%


In [45]:
# Limit the vocabulary that we will use to only words that appear more than the threshold number of times
# New dictionary for words converted to integers

vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= 10 or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to the vocabulary
spec_codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add these codes to the vocabulary
for code in spec_codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers back to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_percentage = round( len(vocab_to_int)/len(word_counts), 4 )*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_percentage))

Total number of unique words: 50231
Number of words we will use: 32794
Percent of words we will use: 65.29%


In [47]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # Create a random embedding for words not in dictionary
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Just to check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

32794


In [48]:
'''Convert words in text to an integer.
   If word is not in vocab_to_int, use UNK's integer.
   Finslly, add EOS token to the end of texts'''

def convert_to_ints(text, word_count, unk_count, eos=False):
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [50]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round( unk_count/word_count, 4 )*100

print("Total number of words in titles:", word_count)
print("Total number of UNKs in titles:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in titles: 888037
Total number of UNKs in titles: 29621
Percent of words that are UNK: 3.34%


In [61]:
# To count the number of time <UNK> appears in a sentence
def unk_counter(sentence):
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [62]:
# Create a data frame of the lengths of sentences from a text

def get_lengths(text):
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
        
    return pd.DataFrame(lengths, columns=['counts'])

In [63]:
lengths_summaries = get_lengths(int_summaries)
lengths_texts = get_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())
print()

Summaries:
            counts
count  7501.000000
mean      9.510599
std       3.184303
min       1.000000
25%       8.000000
50%       9.000000
75%      11.000000
max      46.000000

Texts:
            counts
count  7501.000000
mean    109.878550
std      24.126147
min      15.000000
25%     121.000000
50%     121.000000
75%     121.000000
max     121.000000



In [64]:
# Inspect the length of summaries
print("60% summaries:", np.percentile(lengths_summaries.counts, 60))
print("75% summaries:", np.percentile(lengths_summaries.counts, 75))
print("90% summaries:", np.percentile(lengths_summaries.counts, 90))
print()
# Inspect the length of texts
print("60% texts:", np.percentile(lengths_texts.counts, 60))
print("75% texts:", np.percentile(lengths_texts.counts, 75))
print("90% texts:", np.percentile(lengths_texts.counts, 90))

60% summaries: 10.0
75% summaries: 11.0
90% summaries: 14.0

60% texts: 121.0
75% texts: 121.0
90% texts: 121.0


In [66]:
# Sort the summaries and texts by increasing order of the length of the texts
# Limit the length of summaries and texts based on the min and max ranges.
# Remove articles that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 122
max_summary_length = 15
min_length = 2

unk_text_limit = 5
unk_summary_limit = 0
for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
                sorted_summaries.append(int_summaries[count])
                sorted_texts.append(int_texts[count])
        
# Compare lengths of texts and summaries just to ensure they match
print("Summaries:", len(sorted_summaries))
print("Texts:", len(sorted_texts))

Summaries: 3097
Texts: 3097


BUILDING THE MODEL

In [ ]:
# Create placeholders for the inputs to the model

def model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length